In [1]:
import os
import pandas as pd
import re
import difflib
import json

In [32]:
rashi_masechtot = ['Arakhin', 'Bekhorot', 'Chullin', 'Keritot', 'Menachot', 'Temurah', 'Zevachim', 'Beitzah', 'Chagigah', 'Eruvin', 'Pesachim', 'Megillah', 'Rosh Hashanah', 'Shabbat', 'Sukkah', 'Yoma', 'Gittin', 'Ketubot', 'Kiddushin', 'Sotah', 'Yevamot', 'Bava Batra', 'Bava Kamma', 'Bava Metzia', 'Avodah Zarah', 'Makkot', 'Sanhedrin', 'Shevuot', 'Niddah', 'Berakhot']

mesorat_hashas_path2 = "/Users/ephraimmeiri/gitEtc/Rashi Thesis work/mesorat_hashas_links.json"
mesorat_hashas_file = open(mesorat_hashas_path2)
mesorat_hashas_json = json.load(mesorat_hashas_file)
mesorat_hashas_filtered = [link for link in mesorat_hashas_json if link['refs'][0].split(" ")[0] in rashi_masechtot and link['refs'][1].split(" ")[0] in rashi_masechtot]
shas_refs = [(link['refs'][0],link['refs'][1]) for link in mesorat_hashas_filtered]

In [3]:
def get_rashi_file(masechet):
    file = "/Users/ephraimmeiri/gitEtc/Rashi Thesis work/Rashi plaintext/json undisputed/Rashi on "+masechet+".json"
    if os.path.exists(file):
        with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work//Rashi plaintext/json undisputed/Rashi on "+masechet+".json") as f:
            data = json.load(f)
            return data
    else:
        return None

def get_rashi_ref(ref,join=False):
    masechet,loc1,loc2= parse_ref(ref)
    rashi_file= get_rashi_file(masechet)
    if rashi_file is None:
        return None
    masechet,loc1,loc2= parse_ref(ref)
    text= ""
    with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/Rashi plaintext/json undisputed/Rashi on "+masechet.replace(" ","%20")+".json") as f:
        data = json.load(f)
        index1,line1 = None,None # I'm doing it this way because of the multi-daf behavior
        if ":" not in loc1: # no line
            index1= daf_to_index(loc1)
        else: # has line
            index1= daf_to_index(loc1.split(":")[0])
            line1= int(loc1.split(":")[1])-1
        if loc2==None: # Since loc may have a line range (or just a single line)
            if ":" not in loc1: # no line
                text= data['text'][index1]
            else: # has line
                text= data['text'][index1][line1]
        elif not any(s in loc2 for s in ['a','b']): # loc 2 is a line within this daf
                text= data['text'][index1][line1:int(loc2)]
        else:
            if ":" not in loc2: # no line
                index2= daf_to_index(loc2)
                text= data['text'][index1:index2]
            else: # has
                index2= daf_to_index(loc2.split(":")[0])
                line2= int(loc2.split(":")[1])-1
                text = []
                for i in range(index1,index2+1):
                    # print(i)
                    if(i==index1):
                        text.append(data['text'][i][line1:][0])
                    elif(i==index2):
                        text.append(data['text'][i][:line2+1][0])
                    else:
                        text.append(data['text'][i][0])
    if join:
        while isinstance(text, list):
            # Join the list into a single string
            text = ' '.join(text)
    return text

In [6]:
def parse_ref(ref):
  masechet = ref.split(" ")[:-1]
  masechet= " ".join(masechet)
  loc1= ref.split(" ")[-1].split("-")[0]
  if(len(ref.split(" ")[1].split("-"))==1):
    return masechet,loc1,None
  else:
    loc2= ref.split(" ")[1].split("-")[1]
    return masechet,loc1,loc2
def daf_to_index(ref):
    daf = int(ref[:-1])
    amud= ref[-1]
    index= ((daf-1)*2)
    if(amud=="b"):
      index+= 1
    return index
def get_talmud_file(masechet):
    masechet = masechet.replace(" ","%20")
    with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/talmud_files/"+masechet+".json") as f:
        data = json.load(f)
        return data
def get_talmud_text(ref):
    masechet,loc1,loc2= parse_ref(ref)
    text= ""
    with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/talmud_files/"+masechet.replace(" ","%20")+".json") as f:
        data = json.load(f)
        index1,line1 = None,None # I'm doing it this way because of the multi-daf behavior
        if ":" not in loc1: # no line
            index1= daf_to_index(loc1)
        else: # has line
            index1= daf_to_index(loc1.split(":")[0])
            line1= int(loc1.split(":")[1])-1
        if loc2==None: # Since loc may have a line range (or just a single line)
            if ":" not in loc1: # no line
                text= data['text'][index1]
            else: # has line
                text= data['text'][index1][line1]
        elif not any(s in loc2 for s in ['a','b']): # loc 2 is a line within this daf
                text= data['text'][index1][line1:int(loc2)]
        else:
            if ":" not in loc2: # no line
                index2= daf_to_index(loc2)
                text= data['text'][index1:index2]
            else: # has
                index2= daf_to_index(loc2.split(":")[0])
                line2= int(loc2.split(":")[1])-1
                text = []
                for i in range(index1,index2+1):
                    # print(i)
                    if(i==index1):
                        text.append(data['text'][i][line1:][0])
                    elif(i==index2):
                        text.append(data['text'][i][:line2+1][0])
                    else:
                        text.append(data['text'][i][0])
    while isinstance(text, list):
        # Join the list into a single string
        text = ' '.join(text)
    return text

def get_rashis(link):
    ref1,ref2 = link
    rashi_text1 = get_rashi_ref(ref1)
    rashi_text2 = get_rashi_ref(ref2)
    return rashi_text1, rashi_text2

def get_match(matcher):
    a_start, b_start, = matcher.find_longest_match().a, matcher.find_longest_match().b
    a_end, b_end = a_start , b_start 
    for match in matcher.get_matching_blocks()[:-1]: # "The last triple is a dummy, and has the value (len(a), len(b), 0)."
        if match.a < a_start:
            a_start= match.a
        if match.b < b_start:
            b_start= match.b
        if match.a + match.size > a_end:
            a_end= match.a + match.size
        if match.b + match.size > b_end:
            b_end= match.b + match.size
    return a_start, a_end, b_start, b_end

def sv_in_tal(rashi,tal,threshold=0.4):
    sv= rashi.split(" - ")[0]
    matcher = difflib.SequenceMatcher(None, sv, tal)
    match= matcher.find_longest_match()
    match = get_match(matcher)
    tal_extract = tal[match[2]:match[3]]
    matcher2= difflib.SequenceMatcher(None, sv, tal_extract)
    return matcher2.ratio()>threshold
    # return tal[match[2]:match[3]]

def get_rashis_conservative(link):
    ref1,ref2 = link
    rashi_text1 = get_rashi_ref(ref1)
    rashi_text2 = get_rashi_ref(ref2)
    if rashi_text1 is None or rashi_text2 is None:
        return None, None
    tal_text1 = get_talmud_text(ref1)
    tal_text2 = get_talmud_text(ref2)
    rashis1 = []
    for rashi in rashi_text1:
        if type(rashi)==str and sv_in_tal(rashi,tal_text2):
            rashis1.append(rashi)
        elif type(rashi)==list:
            for r in rashi:
                if sv_in_tal(r,tal_text2):
                    rashis1.append(r)
    rashis2 = []
    for rashi in rashi_text2:
        if type(rashi)==str and sv_in_tal(rashi,tal_text1):
            rashis2.append(rashi)
        elif type(rashi)==list:
            for r in rashi:
                if sv_in_tal(r,tal_text1):
                    rashis2.append(r)           
    return rashis1, rashis2

In [7]:
rashi_output = pd.DataFrame(columns=["masechet1","masechet2","loc1","loc2","txt1","txt2"])
for link in shas_refs:
    rashis = get_rashis_conservative(link)
    if rashis[0] is None or rashis[1] is None:
        continue
    masechet1,loc1 = link[0].split(" ")[0], link[0].split(" ")[1]
    masechet2,loc2 = link[1].split(" ")[0], link[1].split(" ")[1]
    new_row= pd.Series([masechet1,masechet2,loc1,loc2,rashis[0],rashis[1]])
    rashi_output = pd.concat([rashi_output,new_row], ignore_index=True)
with open("/Users/ephraimmeiri/Documents/רשי research docs/rashi CODE AND DATA/rashi_conservative_output.csv", "w") as f:
        rashi_output.to_csv(f)

IndexError: list index out of range

In [9]:
with open("/Users/ephraimmeiri/gitEtc/Rashi Thesis work/shas_refs.json", "w") as f:
    json.dump(shas_refs,f)

In [25]:
import concurrent.futures
import tqdm

def process_link(link):
    try:
        ref1,ref2 = link
        colons1 = ref1.count(":")
        colons2 = ref2.count(":")
        dashes1 = ref1.count("-")
        dashes2 = ref2.count("-")
        len1 = len(ref1.split(" ")[1])
        len2 = len(ref2.split(" ")[1])
        return {(colons1,dashes1,len1), (colons2,dashes2,len2)}
    except Exception as e:
        print(f"Error processing link {link}: {e}")
        return set()
formats = set()
with concurrent.futures.ProcessPoolExecutor() as executor:
    for link in shas_refs:
        formats.update(process_link(link))

In [29]:
formats = set()
print(formats)
examples = [('Ketubot 51a:4',
 'Zevachim 20a:1'),
 ('Sotah 5b:2',
 'Shabbat 120b:11-12'),
 ('Chullin 111a:7',
 'Eruvin 13a:18-13b:2'),
 ('Tosefta Pesachim 7:4',
 'Zevachim 49b:16-17')]
for ex in examples:
    process_link(ex)
print(formats)

set()
set()


In [26]:
formats

{'Ketubot 51a:4',
 'Zevachim 20a:1',
 'Sotah 5b:2',
 'Shabbat 120b:11-12',
 'Chullin 111a:7',
 'Eruvin 13a:18-13b:2',
 'Tosefta Pesachim 7:4',
 'Zevachim 49b:16-17',
 'Yevamot 81a:5-6',
 'Sanhedrin 82a:5',
 'Tosefta Keritot 3:7-4:1',
 'Mishnah Menachot 5:6',
 'Pesachim 3a:3',
 'Shabbat 153b:9',
 'Tosefta Negaim 1:8',
 'Sifra, Tzav, Chapter 12 14',
 'Shabbat 15a:5',
 'Bekhorot 51a:10-11',
 'Tosefta Zavim 5:9',
 'Sukkah 56a:5-6',
 'Tosefta Gittin 1:2',
 'Eruvin 9a:8',
 'Niddah 48b:2',
 'Tosefta Sanhedrin 2:4-5',
 'Yoma 49b:9',
 'Shabbat 57b:3-4',
 'Yevamot 31a:2',
 'Otzar Midrashim, Chuppah of Eliyahu, Chuppat Eliyahu Rabbah 44',
 'Niddah 68b:3',
 'Zevachim 82b:10-13',
 'Zevachim 3b:11',
 'Bekhorot 17a:21',
 'Shabbat 146a:3-4',
 'Chullin 57a:8',
 'Eruvin 31b:4-5',
 'Menachot 69a:8',
 'Ketubot 69b:11-70a:2',
 'Sifra, Emor, Chapter 1 1',
 'Tosefta Chullin 1:10',
 'Beitzah 27b:5',
 'Tosefta Shabbat 1:8',
 'Yoma 70a:11',
 'Shevuot 34b:5-6',
 'Sukkah 27a:4',
 'Kiddushin 32b:8',
 'Eruvin 94b:6

In [31]:
len(shas_refs)

14720

In [33]:
len(shas_refs)

7989

In [34]:
for link in shas_refs:
    ref1,ref2 = link
    if ":" not in ref1:
        print(ref1)
    if ":" not in ref2:
        print(ref2)

In [36]:
for link in shas_refs:
    ref1,ref2 = link
    if "-" not in ref1:
        print(ref1)
    if "-" not in ref2:
        print(ref2)

Shabbat 75a:6
Sukkah 33b:11
Chullin 108a:8
Menachot 23a:1
Menachot 23a:1
Shabbat 107a:3
Sukkah 6b:12
Yevamot 53a:4
Yevamot 112b:18
Yevamot 96b:9
Niddah 67b:2
Yevamot 78a:20
Berakhot 40a:11
Shabbat 127a:13
Yevamot 76b:3
Yevamot 78a:13
Ketubot 22b:2
Yevamot 88b:3
Eruvin 95a:5
Yoma 49a:1
Chagigah 3b:10
Megillah 10a:7
Zevachim 12b:20
Zevachim 16a:4
Keritot 20b:4
Kiddushin 64b:6
Bekhorot 53b:17
Temurah 5a:8
Sanhedrin 19a:15
Niddah 61b:13
Chullin 27a:18
Yoma 25b:16
Yevamot 17a:5
Sanhedrin 45b:12
Sanhedrin 67b:4
Pesachim 25b:3
Menachot 28a:7
Menachot 34a:16
Sanhedrin 18b:14
Beitzah 11a:20
Bekhorot 37a:3
Yevamot 87b:15
Yevamot 95b:7
Yevamot 108a:16
Yevamot 35b:1
Menachot 76a:2
Niddah 63a:11
Sanhedrin 16b:14
Ketubot 104a:10
Ketubot 55a:2
Pesachim 16a:16
Pesachim 20a:4
Ketubot 100b:6
Menachot 74a:8
Eruvin 16b:20
Gittin 41a:12
Gittin 19b:12
Gittin 4b:8
Shevuot 36b:12
Shevuot 38a:20
Kiddushin 49b:6
Sanhedrin 24a:14
Chullin 34b:3
Niddah 7a:8
Gittin 76b:9
Ketubot 2b:10
Shevuot 29a:10
Shevuot 39a:7
B

In [42]:
def parse_ref(ref):
  masechet = ref.split(" ")[:-1] # Get all but the last element
  masechet= " ".join(masechet)
  colon_count = ref.count(":")
  if "-" not in ref:
    return masechet,ref.split(" ")[-1],None
  elif colon_count == 1:
    loc1,loc2= ref.split(" ")[-1].split("-")
    return masechet,loc1,loc2
  else:
    loc1,loc2= ref.split(" ")[-1].split("-")
    return masechet,loc1,loc2
  

In [43]:
parse_ref("Sanhedrin 103b:6")

('Sanhedrin', '103b:6', None)

In [44]:
parse_ref("Bava Kamma 103b:6-104a:7")

('Bava Kamma', '103b:6', '104a:7')